# ALICIA
## Watson
#### Speech to Text, Text to Speech y Assistant

In [13]:
from __future__ import print_function
from IPython.display import clear_output
from datetime import datetime
from pytz import timezone
import json
import random
import time

#LIBRERIAS GRABACION
import pyaudio
import wave

#LIBRERIAS REPRODUCIR WAV
from pydub import AudioSegment
from pydub.playback import play

#LIBRERIAS Speech to Text
#from __future__ import print_function
from os.path import join, dirname
#import json
from ibm_watson import SpeechToTextV1
from ibm_watson.websocket import RecognizeCallback, AudioSource
import threading

#LIBRERIAS Assistant
#from __future__ import print_function
#import json
from ibm_watson import AssistantV2

#LIBRERIAS Text to Speech
#from __future__ import print_function
#import json
#from os.path import join, dirname
from ibm_watson import TextToSpeechV1
from ibm_watson.websocket import SynthesizeCallback

In [25]:
#CONFIGURACION

__HABILITAR_TTS = False
__HABILITAR_STT = False

__FOLDER = './resources/'

__FILE_STT = 'stt_input.wav'
__FILE_TTS = 'tts_output.wav'
__FILE_BEEP = 'beep.wav'
__FILE_BEEP_FINISH = 'beep_finish.wav'

__APIKEY_TEXT_TO_SPEECH= 'E4ynpXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'
__APIKEY_SPEECH_TO_TEXT= 'bI5_XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'
__APIKEY_ASSISTANT = 'g2_XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'
__ASSISTANT_ID = '40e8e90c-XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'

In [15]:
topicos = ["Aceleración: Cambio de la velocidad en el tiempo. Siempre que la velocidad de un cuerpo cambia al transcurrir el tiempo, ya sea porque cambia su magnitud o su dirección o ambas cosas a la vez, se puede afirmar que existe aceleración.","Alfa: Es la primera letra del alfabeto griego. En cálculo o ecuaciones diferenciales representa a números infinitesimales, en geometría y trigonometría representa a ángulos y en física es una constante.","Álgebra: Es una rama de la matemática que emplea números, letras y signos.  Cada letra o signo representa simbólicamente un número u otra entidad matemática.","Ángulo: Un ángulo es la parte del plano comprendida entre dos semirrectas que tienen el mismo punto de origen o vértice.","Área: Es una medida de la extensión de una superficie, expresada en unidades de medida denominadas “Unidades de Superficie”. Equivale a 100 metros cuadrados. Para la geometría, un área es la superficie comprendida dentro de un perímetro, que se expresa en unidades de medidas que son conocidas como superficiales. Cuando alguno de los signos representa un valor desconocido se llama incógnita.","Beta: Es la segunda letra del alfabeto griego. En cálculo o ecuaciones diferenciales representa a números infinitesimales, en geometría y trigonometría representa a ángulos y en física es una constante.","Binomio: Un binomio consta únicamente de dos términos unidos por los signos “+” o “-“ ","Bisectriz: La bisectriz de un ángulo es el lugar geométrico de los puntos que equidistan de los lados de un ángulo.","Biyectiva: Es la función que al mismo tiempo es inyectiva y sobreyectiva a la vez, es decir si todos los elementos del conjunto de salida tienen una imagen distinta en el conjunto de llegada, y a cada elemento del conjunto de llegada le corresponde un elemento del conjunto de salida.","Cartesiano: Es el plano que está formado por dos rectas numéricas, una horizontal y otra vertical que se cortan en un punto. Tiene como finalidad describir la posición de puntos, los cuales se representan por sus coordenadas o pares ordenados.","Cateto: Un cateto es cualquiera de los dos lados menores de un triángulo rectángulo, los que conforman el ángulo recto.","Cilindro: Cuerpo geométrico que se obtiene por la rotación de un rectángulo en torno a uno de sus lados.","Circunferencia: Una circunferencia es el lugar geométrico de los puntos de un plano que equidistan de otro punto fijo y coplanario llamado centro en una cantidad constante llamada radio.","Cociente: Es el resultado que se obtiene al dividir una cantidad por otra, y que expresa cuantas veces está contenido el divisor en el dividendo.","Coeficiente: Constante o número que multiplica a una variable en una expresión algebraica.","Constante: Una constante es una cantidad que tiene un valor fijo en un determinado cálculo, proceso o ecuación. Esto quiere decir que la constante es un valor permanente que no puede modificarse.","Continuidad: Una función continua es aquella para la cual intuitivamente, para puntos cercanos del dominio se producen pequeñas variaciones en los valores de la función. Si la función no es continua se dice que es discontinua.","Cosecante: Es la razón trigonométrica inversa del seno.","Coseno: En trigonometría el coseno de un ángulo en un triángulo rectángulo, e define como la razón entre el cateto adyacente a ese ángulo y la hipotenusa.","Cotangente: Es la razón trigonométrica inversa de la tangente.","Curva: Es una línea continua de una dimensión, que varía de dirección paulatinamente.","Decágono: Es un polígono que consta de diez lados.","Decaedro: Poliedro de diez caras.","Decímetro: Medida de longitud equivalente a la décima parte del metro.","Delta: La diferencia entre dos valores próximos de una magnitud y varias funciones y operadores.","Denominador: Indica las partes iguales en que se divide la unidad en una fracción.","Determinante: En Matemáticas se define el determinante como una forma multilineal alternada de un cuerpo. Esta definición indica una serie de propiedades matemáticas y generaliza el concepto de determinante haciéndolo aplicable en numerosos campos. Sin embargo, el concepto de determinante o de volumen orientado fue introducido para estudiar el número de soluciones de los sistemas de ecuaciones lineales.","Diferencia: Es el resultado de la resta o sustracción. La resta consiste en quitar una cierta cantidad de una cifra, la cantidad resultante es la diferencia.","Diferencial: Se refiere a un cambio en la linearización de una función.","Dividendo: Es el nombre matemático del primer término de la división.","Divisor: Es el término matemático del segundo término de la división.","Dominio: El dominio de una función es el conjunto de existencia de ella misma, es decir, los valores para los cuales la función está definida. El conjunto de todos los posibles valores de ingreso que la función acepta.","Elipse: Es el lugar geométrico de todos los puntos de un plano, tales que la suma de las distancias a otros dos puntos fijos llamados focos es constante.","Eneágono: Es un polígono de nueve lados.","Épsilon: Es la quinta letra del alfabeto griego. Suele designar a cantidades que tienden hacia cero, en particular en el estudio de los límites y de la continuidad.","Equilátero: Triángulo cuyos lados y ángulos son de igual medida.","Escaleno: Es el triángulo que tiene sus tres lados desiguales.","Exponencial: Es la función donde e es el número de Euler. Esta función tiene por dominio de definición el conjunto de  los números reales, y tiene la particularidad de que su derivada es la misma función. ","Exponente: Se refiere al número de veces que se debe multiplicar por sí misma la base de una potencia.","Factorial: Producto obtenido al multiplicar un número pósitivo dado, por todos los enteros positivos inferiores a ese número.","Factorización: La factorización de un polinomio es el mecanismo que permite expresar a ese polinomio como el producto de dos o más polinomios.","Fracción: Representación de un número que consta de un valor llamado numerador (representado en la parte superior) y otro llamado denominador (representado en la inferior).","Función: Es una asociación que se establece entre los elementos de un conjunto A y los de un conjunto B, de manera que a cada elemento del conjunto A se le asocia un solo elemento del conjunto B","Geometría: Es una rama de las matemáticas que se ocupa del estudio  de medidas, propiedades y relaciones entre puntos, líneas, ángulos, superficies y sólidos.","Hectómetro: Medida de longitud equivalente a 100 metros.","Heptaedro: Poliedro de siete caras.","Hipérbola: Una hipérbola es el lugar geométrico de los puntos de un plano tales que el valor absoluto de la diferencia de sus distancias a dos puntos fijos, llamados focos, es igual a la distancia entre los vértices, la cual es una constante positiva.","Hipotenusa: Es el lado de mayor longitud de un triángulo rectángulo, y el lado opuesto al ángulo recto. La medida de la hipotenusa puede ser hallada mediante el teorema de Pitágoras, si se conoce la longitud de los otros dos lados denominados catetos.","Icosaedro: Poliedro de veinte caras.","Indeterminación: Se refiere a que la aplicación de las propiedades de los límites no son válidas, sin embargo no significa que el límite no exista o no se pueda determinar.","Infinito: Cantidad sin límite, la misma puede ser numerable o no numerable. Es un signo en forma de ocho tendido que sirve para expresar un valor mayor que cualquier cantidad asignable.","Integral: La integral es la operación inversa respecto de la derivada. La integral calcula el área debajo de una curva.","Intervalo: Se llama intervalo al conjunto de números reales comprendidos entre otros dos dados; a y b que se llaman extremos del intervalo. Es un conjunto comprendido entre dos valores.","Inyectiva: Es la función donde a cada elemento del dominio le corresponde una imagen distinta en el codominio. En términos matemáticos una función es inyectiva si a cada valor del conjunto X (dominio) le corresponde un valor distinto en el conjunto Y (imagen).","Límite: El límite describe la tendencia de una sucesión o una función, a medida que los parámetros de esa función se acercan a determinado valor.","Logaritmo: El logaritmo de un número en una base determinada, es el exponente al cual hay que elevar la base para obtener dicho número.","Matriz: Una matriz es un arreglo, en filas y columnas, de números que son llamados coeficientes.","Mixto: Es el número compuesto de un número entero y una fracción.","Múltiplo: Un múltiplo de un número es otro número que lo contiene un número entero de veces. En otras palabras, un múltiplo de n es un número tal que, dividido por n, da por resultado un número entero.","Numerable: Es el conjunto con el que se puede establecer una correspondencia biyectiva con el conjunto de los números naturales.","Numerador: Indica las partes iguales que se toman de la unidad en una fracción.","Obtusángulo: Triángulo que tiene un ángulo obtuso.","Ordenada: Segunda componente del par ordenado (x,y) que determinan un punto del plano en un sistema de coordenadas cartesianas.","Ortoedro: Paralelepípedo cuyas bases son rectángulos y sus aristas laterales perpendiculares a las básicas.","Óvalo: Curva cerrada con dos ejes de simetría perpendiculares entre sí y compuesta de varios arcos de circunferencia tangentes entre sí.","Parábola: Se define como el lugar geométrico de los puntos de un plano que equidistan de una recta (eje o directriz) y un punto fijo llamado foco.","Pentadecágono: Polígono de quince lados","Pentágono: Es un polígono que consta de cinco lados y cinco vértices.","Perímetro: Se refiere a la longitud de una curva cerrada.","Perpendicular: Rectas que se cortan formando ángulos rectos.","Pi: Número irracional que corresponde a la razón entre la longitud de la circunferencia y su diámetro.","Pirámide: Cuerpo geométrico que tiene como base un polígono cualquiera y como caras laterales triángulos con un vértice común.","Polígono: es una figura plana compuesta por una secuencia finita de segmentos rectos consecutivos no alineados. Estos segmentos son llamados lados, y los puntos en que se interceptan se llaman vértices. El interior del polígono es llamado cuerpo.","Polinomio: expresión algebraica compuesta de dos o más términos llamados monomios unidos por los signos más o menos.","Porcentaje: Es una razón cuyo consecuente es 100.","Potencia: Una potencia es la manera abreviada en la que escribimos una multiplicación en la que todos sus factores son iguales.","Primos: Son aquellos números que tienen la capacidad de poseer únicamente dos divisores, el mismo número y el 1, que es divisor de todo número. ","Producto: Es el resultado que se obtiene de multiplicar dos o más cantidades.","Q: Símbolo con el que se representa el conjunto de los números racionales.","Quebrado: Término con el que también se designa una fracción.","Radián: Unidad de medida de ángulos que equivale a un ángulo que con el vértice en el centro de la circunferencia subtiende un arco de longitud igual al radio de esta circunferencia.","Radio: Segmento que une el centro con un punto cualquiera de la circunferencia.","Raíz: Cantidad que ha de multiplicarse por sí misma una o más veces para obtener un número determinado.","Rango: Es el conjunto de todos los valores de salida de una función.","Reales: Es el conjunto de números resultante de la unión de los racionales con los irracionales.","Recíproco: Corresponde al valor inverso de un número, de manera tal que al efectuar el producto entre ambos, resulta 1.","Rectángulo: Un rectángulo es un polígono de cuatro lados, en donde cada ángulo es un ángulo recto, es decir de 90 grados.","Secante: Es la razón trigonométrica inversa del coseno.","Seno: En trigonometría el seno de un ángulo en un triángulo rectángulo, se define como la razón entre el cateto opuesto y la hipotenusa.","Sobreyectiva: Una función es sobreyectiva si cada elemento del codominio tiene preimagen en el dominio, de la función. ","Subconjunto: Conjunto que forma parte de otro conjunto dado.","Suma: Consiste en añadir dos números o más para obtener una cantidad total.","Tangente: En trigonometría la tangente de un ángulo en un triángulo rectángulo se define como la razón entre el cateto opuesto y el adyacente.","Teorema: Se llama Teorema a toda afirmación matemática importante que es demostrada de manera rigurosa, irrefutable. Un teorema es una afirmación que puede ser demostrada dentro de un sistema formal. Demostrar teoremas es un asunto central en la matemática.","Trapecio: Cuadrilátero con un par de lados paralelos.","Trigonometría: La trigonometría es una rama de la matemática, cuyo significado etimológico es la medición de los triángulos.","Trinomio: Expresión algebraica de tres términos.","Variable: Una variable es la expresión simbólica representativa de un elemento no especificado, cuyo valor puede ser modificado.","Vector: Un vector es todo segmento de recta dirigido en el espacio. Cada vector posee unas características que son; origen, módulo, dirección, sentido.","Derivada: La derivada de una función es una medida de la rapidez con la que cambia el valor de dicha función según cambie el valor de su variable independiente."]
animales = ["Ácaro: El ácaro es una clase de arácnidos. La mayoría son diminutos: miden menos de un milímetro. Se estima que más de cincuenta mil especies ya han sido descritas y que existen un millón o más en la actualidad.","Abeja: La abeja es un insecto volador. Son polinizadores, lo que significa que van de flor a flor para obtener su néctar, de lo que se alimentan y de pasada se llevan el polen. Su color es, por lo general, amarillo y negro a rayas, aunque existen varias especies diferentes. Tienen un aguijón con el que pueden picar animales y personas, pero mueren poco después por la pérdida del mismo.","Abejorro: Insecto parecido a la abeja pero de cuerpo más gordo y de hasta 3 cm de largo, cubierto de vello oscuro y con la trompa muy desarrollada, que emite un zumbido intenso al volar; vive en enjambres poco numerosos debajo del musgo o de las piedras.","Águila: El águila es un ave grande y rapaz, es un ave de rapiña. Son carnívoras.","Alacrán: Los escorpiones o alacranes son un orden de arácnidos provistos de un par de apéndices en forma de pinza y una cola acabada en un aguijón.","Alce: El alce es un ciervo grande. Esta especie habita en el norte de Europa, Asia y en América del Norte, en zonas con lagos, pantanos y humedales. También residen en las montañas.","Alpaca: La alpaca es un camelido altoandino localizado en Argentina, Bolivia, Chile y Perú. Es de aptitud pecuaria; así se crían para carne y fibra. \n\nUno de los productos cárnicos tradicionales obtenido de la alpaca es el charqui ","Araña: Las arañas construyen telarañas estratégicamente desde un determinados sitio hasta otro. Sus telarañas les permiten que los insectos que se aproximen a ella queden pegados en su telaraña y después comérselos.","Ardilla: Una ardilla es un mamífero, particularmente, un roedor. Es omnívoro, come piñas, frutos, semillas, cortezas e incluso insectos, huevos y aves. Vive en los árboles y en el suelo. La ardilla construye nidos para ocultarse y tener descendencia. En invierno, la ardilla vive del stock de invierno. No hiberna sino que se mantiene activa consumiendo lo que ha ido almacenando en diferentes oquedades de los árboles y las rocas.","Avestruz: El avestruz es un ave de origen africano, que vive en la tierra y no puede volar. Es la más grande, más pesada y más rápida de las aves en el mundo. La especie no está amenazada y se encuentra en estado salvaje en África.","Burro: Un burro es un animal mamífero. Su nombre científico es Equus asinus. Estos animales son dóciles y amistosos, y si reciben un buen trato pueden rápidamente forjar amistad con los niños. Muestran gran curiosidad y tienen una disposición juguetona. En sitios abiertos les gusta revolcarse en la arena, lo que les permite librarse de parásitos que se esconden en su pelaje.","Ballena: Las ballenas son mamíferos marinos que viven en los océanos. Como otros mamíferos, las ballenas respiran oxígeno del aire, tienen una cantidad pequeña de pelo y son de sangre caliente.","Buey: Buey es el nombre que se le da al macho bovino (toro) castrado, dedicado específicamente al engorde y sacrificio para utilizar su carne y su piel, y antiguamente también a la realización de tareas agrícolas como tirar de arados y carros.","Búho: El búho es un ave del orden strigiformes.","Caballo: El caballo es un mamífero que hace parte de la familia de los équidos. Es un animal domesticado por el hombre, y utilizado para numerosas actividades: agricultura, transporte, deportes, y guerra, entre otras.","Cabra: La cabra es un mamífero rumiante. Originaria de Asia, se haya muy extendida por todo el mundo. ","Cacatúa: La cacatua es un género de aves psittaciformes. Estas aves provienen de Asia y Oceanía, y se encuentran desde las islas Filipinas y el este de Wallacea hasta las Islas Salomón y el sur de Australia. Es de la familia Cacatuidae, la cual tiene 21 especies de aves. ","Calamar: Los calamares son cefalópodos. Los calamares son carnívoros, pero también son cazados como presas. El calamar más grande, el calamar gigante y el calamar colosal, son comidos por los cachalotes y los tiburones durmientes. ","Camello: Un camello (nombre científico: Camelus) es un animal mamífero.","Cangrejo: Cangrejo puede referirse a:","Canguro: Los canguros son mamíferos de la familia de los marsupiales. Son herbívoros (es decir, comen vegetales) y miden entre 1,50 y dos metros de alto. Tienen una bolsa, la cual sirve para que la cría viva allí hasta que puede alimentarse por sí misma.","Caracol: El caracol  es un molusco gasterópodo. Su cuerpo blando, sin esqueleto, está protegido de los depredadores por una concha enrollada en la que puede  hibernar. ","Castor: El castor es un animal mamífero que vive en lugares con agua. Se caracteriza por construir diques a partir de árboles, a los cuales cortan para obtener sus troncos. ","Cebra: Las cebras son animales de la familia biológica de los équidos que viven en África, son parientes muy cercanos de los caballos y los burros. Las cebras tienen rayas blancas y negras y se piensa que les sirve de camuflaje.","Cerdo: El cerdo, chancho o puerco es un animal omnívoro, doméstico. Su domesticación se remonta a miles de años. En China se cría el cerdo desde hace unos 6.000 años.","Güepardo: Un güepardo es un animal mamífero que habita en África. El güepardo es muy rápido ya que puede correr de 95 a 120 kilómetros por hora en carreras de 400 a 500 metros, lo que lo convierte en el animal terrestre más veloz del mundo. Su presa principal es la gacela.","Ciervo: El ciervo es un mamífero perteneciente a los cérvidos. Se trata de un herbívoro: porque se alimenta de plantas (hierba, hojas, cortezas) y vive en los bosques y en ocasiones en los prados de Europa, del Norte de África, de Asia, de Norteamérica y de América del Sur. Normalmente no es difícil de observar, pero hay que tener mucho cuidado y no hacer ruido. El ciervo hembra es la cierva; el ciervo pequeño se llama cervato.","Ciervo volante: El ciervo volante es una especie de coleóptero escarabeiforme perteneciente a la familia Lucanidae. Se lo considera el escarabajo más grande de Europa. Su tamaño total varía entre los 3 y 10 cm en los machos y los 2,8 y 5,4 cm en las hembras. Presentan un notable dimorfismo sexual; los machos son mayores que las hembras y poseen las mandíbulas mucho más desarrolladas que utilizan para la lucha; tienen una forma que recuerda los cuernos de un ciervo, de donde deriva su nombre común.","Cigüeña blanca: Las aves típicamente que caminan son zancudas. Y así capturan sus presas en el fondo del agua. En materia de clasificación eran parientes de espátulas, garzas, pelícanos, ave martillo y  pico zapato. En la reciente clasificación los ciconiformes quedan como parafilético de sus tradicionales parientes de afinidad.","Coatí: Los coatíes, también llamados cusumbos, cusumbosolos, cuchuchos, guaches, gatos solos, mípalos o pizotes, son dos especies de pequeños mamíferos omnívoros americanos de la superfamilia Canoidea y la familia de los prociónidos.","Cocodrilo: El cocodrilo es un reptil grande, que vive en las regiones tropicales de África, Asia, América y Australia. Aparecieron por primera vez durante el Eoceno, unos 55 millones de años. Su piel es escamosa, dura y seca. Llegan a vivir unos 50 a 80 años.","Colibrí: El colibrí o zunzún es el ave más pequeña del mundo. Su nombre científico es Trochilinae. Existen más de 343 especies identificadas, y viven en muchas regiones del continente americano. Se conoce como la segunda especie de aves más grande del mundo, por su gran número de especies.","Cóndor: Cóndor es el nombre de dos especies de buitres del Nuevo Mundo. Son las aves terrestres voladoras más grandes del Hemisferio Occidental. Son:","Conejo: Un conejo es un animal mamífero del orden de los Lagomorfos, y el único del género Oryctolagus. Mide unos cuatro decímetros de largo, comprendida la cola. Tiene pelo espeso de color ordinariamente gris, orejas tan largas como la cabeza, patas posteriores más largas que las anteriores, aquéllas con cuatro dedos y éstas con cinco, y cola muy corta. Vive en madrigueras, se domestica fácilmente, su carne es comestible y su pelo se emplea para fieltros y otras manufacturas.","Cucaracha: Los blatodeos, conocidos también como cucarachas, cutias o baratas, son un orden de insectos hemimetábolos paurometábolos de cuerpo aplanado, que miden entre 3 cm a 7,5 cm. Incluye también a las termitas, infraorden Isoptera.","Cuervo: El cuervo grande es una especie de ave paseriforme de la familia Corvidae. Presente en todo el hemisferio septentrional, es la especie de córvido con la mayor superficie de distribución. A pesar de ello, la corneja negra, de menor tamaño aunque muy similar en su aspecto morfológico externo, es muy abundante en sus propias áreas de distribución, por lo que a menudo se confunde a las cornejas negras con cuervos grandes.","Delfín: El delfín común oceánico es una especie de cetáceo  de la familia Delphinidae propia de aguas tropicales y subtropicales. ","Dinosaurio: Una de las principales características de los dinosaurios es la propiedad de tener las patas situadas en posición vertical por debajo del cuerpo, como los mamíferos, y no hacia los costados, como la mayor parte de los reptiles.","Dragón de Komodo: Dragón de Komodo, también llamado monstruo de Komodo y varano de Komodo, es una especie de saurópsido de la familia de los varánidos, endémico de algunas islas de Indonesia central. Es el lagarto de mayor tamaño del mundo, con una longitud media de dos a tres metros y un peso de unos 70 kg.","Dodo: El dodo es un ave extinta. Medraba solo en los bosques de la isla de Mauricio y no podía volar. De un metro de largo y pesaba alrededor de 20 libras. Su envergadura era su única defensa, hasta la llegada de los colonos holandeses.","Estrella de mar: La estrella de mar es un animal marino que tiene forma de estrella de cinco puntas o más. Pertenece al grupo de los Equinodermos o animales con piel espinosa.","Dromaius: Dromaius es un género de aves Casuariiformes de la familia Dromaiidae conocidas coloquialmente como emú. Son aves grandes, no voladoras. Su hábitat natural se encuentra en el continente de Oceanía; en la actualidad solo sobrevive una especie, emú común, que es la segunda ave de mayor tamaño, después del avestruz.","Elefante: El elefante es un animal herbívoro terrestre, el más grande que existe. Viven en manadas, que a veces tienen cientos de integrantes. Tiene piel gruesa de color gris o amarronado y una muy buena memoria.","Escorpión: El escorpión es un orden de arácnidos que tienen un par de apéndices en forma de pinza, y una cola que acaba en un aguijón venenoso. Actualmente se conocen aproximadamente mil cuatrocientas especies en todo el mundo.","Equidna: Un equidna es un mamífero ovíparo de cuerpo rechoncho revestido de pelo pardo oscuro y espinas blancas, que tiene un hocico afilado, lengua larga, ojos pequeños, patas robustas, cola muy corta y uñas largas y fuertes; es insectívoro y vive en regiones de matorrales de Oceanía.","Phasianus colchicus: Faisán común o faisán vulgar","Foca: Las focas  son mamíferos que pasan la mayor parte del tiempo en los océanos. Son más comunes en las zonas árticas y antárticas aunque también habitan otras regiones costeras, con la excepción de las zonas tropicales.","Gallus gallus domesticus: El gallo y la gallina son la subespecie doméstica de la especie Gallus gallus, una especie de ave galliforme de la familia Phasianidae procedente del sudeste asiático. Los nombres comunes son: gallo, para el macho; gallina, para la hembra, y pollo, para los subadultos. Es el ave más numerosa del planeta.Los gallos y gallinas se crían principalmente por su carne y por sus huevos.","Gato: Es un mamífero carnívoro de la familia Felidae. Es una subespecie domesticada por la convivencia con el ser humano.","Gorila: Los gorilas son primates herbívoros que habitan los bosques de África central. Es el más grande de los primates vivos. Su ADN es un 97%–98% igual al humano, siendo el más cercano a éste después de las dos especies de chimpancé.","Guacamayo: El guacamayo también conocido como papagayo es una especie de loro que vive en América del Sur. Desde Panamá hasta el norte de Paraguay y Bolivia, pasando por toda la cuenca amazónica. Se reproducen fácilmente en cautividad; en libertad viven en grupos de 25-30 ejemplares. Tienen poderosos picos y colas muy largas.","Guanaco: El guanaco, es un camélido originario de América del Sur. Es un animal salvaje, en oposición a la llama, de huesos finos, con una altura aproximada de 1,60 metros y cerca de 91 kilogramos de peso. Los guanacos jóvenes son llamados chulengos. Los guanacos se encuentran generalmente en pequeños rebaños o grupos familiares poco estructurados. Su mayor depredador es el puma","Gusano: Un gusano es un animal pequeño y alargado que se arrastra sobre la tierra y puede vivir en varios hábitats: bajo el suelo horadándolo, en otros animales o en el agua. No tiene columna vertebral.","Hámster: Los hamsters son pequeños roedores domésticos. En algunos laboratorios, suelen utilizarlos para hacer experimentos.","Hipopótamo: Los hipopótamos son los animales mamíferos que forman la familia de los hipopotámidos, dentro del orden de los artiodáctilos. Dentro de esa familia se clasifican el hipopótamo común y el hipopótamo pigmeo, además de otras especies extintas.","Hormiga: La hormiga es un insecto social que pertenece a la orden de los himenópteros. Existen alrededor de 20 000 especies de hormigas en el mundo.","Iguana: Las iguanas son animales pertenecientes a la familia de los lagartos. Son reptiles y algunas especies pueden tenerse como mascotas. Viven entre siete y diez años.","Jabalí: El jabalí es un animal mamífero también conocido como cerdo salvaje. Tiene un tamaño mediano, cabeza grande, patas cortas y un pelaje espeso. En varios países es considerado un manjar y consumido por los seres humanos.","Jaguar: El jaguar, yaguar o yaguareté es un carnívoro félido. Es el mayor félido de América y el tercero del mundo, después del tigre y el león. ","Jirafa: La jirafa es la más alta de todas las especies vivientes de animales terrestres. Pueden tener hasta 5,8 metros de altura y pesan entre 750 y 1600 kilogramos. Su nombre proviene del árabe y significa \"alta\".","Koala: Un koala es un animal marsupial mamífero que vive en Australia. Tiene cuatro patas y es herbívoro.","Lagarto: Los largartos son una especie de reptiles que incluyen la mayoría de los reptiles, como los camaleones, las lagartijas y las iguanas.","León: El león es un mamífero carnívoro. Al igual que los gatos, hace parte de los felinos.","Leopardo: El leopardo es un mamífero carnívoro del género Panthera, que incluye tambien al león, al tigre y al jaguar, y como ellos también puede rugir. ","Lince: Los linces conforman cuatro especies de felinos salvajes caracterizadas por su tamaño medio, fuertes patas, largas orejas, cola corta y cuerpo más o menos moteado.","Lobo: El lobo es un animal mamífero perteneciente a la familia de los cánidos.","Llama: La llama es un mamífero doméstico, abundante en la puna o altiplano de los Andes de Argentina, Bolivia, Chile, Ecuador y Perú.","Mamut: El mamut es uno de los antepasados del elefante, actualmente extintos. Este animal tiene características idénticas al elefante, excepto que tiene pelo, esto se debe a que aquellos mamuts vivían en sitios casi congelados, todos murieron por eso. Actualmente, los elefantes no tienen pelo debido a la subida indómita de calor.","Mariposa: La mariposa es un insecto perteneciente a la familia de los lepidópteros.","Monstruo de Gila: El monstruo de Gila es un gran lagarto que vive en el Sur de los Estados Unidos, y en el Norte de Mexico, en los desiertos. Es uno de los pocos lagartos venenoso del mundo, pero no es peligroso para el ser humano.","Mono: Los monos son animales que forman parte de los primates, mamíferos adaptados a la vida en los árboles. ","Mosca: La mosca es un insecto díptero, es decir, tiene solamente un par de alas funcionales. El otro par está atrofiado (crece muy poco) y sirve para mantener el equilibrio del cuerpo durante el vuelo . La mosca es un insecto con metamorfosis: va desde el estado de huevo a la de gusano, después pupa, y finalmente una mosca. La mosca adulta pone huevos y todo comienza de nuevo.","Murciélago: El murciélago es un mamífero nocturno cuyas extremidades superiores se desarrollaron como alas. Son los únicos mamíferos capaces de volar. Para conseguirlo, han desarrollado una serie de caracteres destinados a permitir el vuelo; excepto el pulgar, todos los dedos de las manos están particularmente alargados y sostienen una fina membrana de piel, flexible y elástica, que garantiza sujetarse en el aire. Se han extendido por casi todo el mundo.","Nutria: Las nutrias o lutrinés son mamíferos carnívoros acuáticos que forman una  subfamilia, la de los mustélidos y son excelentes  nadadores.","Ñandú: El ñandú es un ave corredora, parecida al avestruz, que solo se encuentra  en Sudamérica. Habita las regiones pampeanas, el Chaco y Mesopotamia en Argentina; Uruguay; centro-sur de Brasil y este de Paraguay y Bolivia. El ñandú petiso de la Patagonia habita en la Región Patagónica de Argentina y Chile y sur de Mendoza. En Chile se encuentra en la región de Aysén y de Magallanes. El ñandú de la Puna se distribuye por el sur de Perú y Bolivia y nordeste de Chile y Argentina.","Ñu: El ñu es un animal mamífero que pertenece a la familia de los bóvidos. Son herbívoros y los machos adultos tienen cuernos sobre sus cabezas, como todos los antílopes.","Oso polar: El oso polar u oso blanco es un mamífero carnívoro que habita en la región ártica, cerca del Polo Norte. A pesar de vivir en una de las zonas con menos temperatura en el mundo, los osos aguantan el frío, gracias a su pelaje y su grasa corporal, que les sirve como un gran abrigo. Se alimentan de muchos animales árticos, en especial de las focas, y ocasionalmente, de renos. Los osos polares no toman agua, ya que en su ambiente es salada y ácida.","Oso: Animal perteneciente a las distintas especies de la familia de los úrsidos, que incluyen entre otras al oso pardo, al oso polar, al oso panda o panda gigante y al oso frontino u oso de anteojos (única especie sudamericana existente en la actualidad).","Leopardus pardalis: Leopardus pardalis es una especie de mamífero carnívoro de la familia Felidae. Se encuentra ampliamente distribuida en América, donde se diferencia en numerosas subespecies.","Oruga: Una oruga es una larva de mariposa (insectos lepidópteros).","Ornitorrinco: El ornitorrinco es un animal mamífero semiacuático que vive en Australia y la isla de Tasmania. Es uno de los pocos mamíferos actuales que pone huevos en lugar de dar a luz crías vivas.","Oveja: La oveja es un animal mamífero que se caracteriza por tener el cuerpo cubierto de lana, la cual puede extraerse y usarse para fabricar prendas de ropa y cobijas, entre otras cosas.","Pajaro: Las aves son animales vertebrados, de sangre caliente, que caminan, saltan o se mantienen solo sobre las extremidades posteriores, mientras que las extremidades anteriores han evolucionado hasta convertirse en alas que, al igual que muchas otras características anatómicas únicas, les permiten, en la mayoría de los casos, volar, si bien no todas vuelan. Tienen el cuerpo cubierto de plumas. Para reproducirse ponen huevos que incuban hasta su eclosión.","Paloma: La paloma es un ave de la familia Columbidae dentro del orden Columbiformes. Existen aproximadamente 308 especies especies distintas de palomas. Son parientes del extinto dodo.","Pantera: La pantera es un felino que habita en los bosques de Asia y África. Pesa entre 50 y 100 kg. Buen cazador, permanece al acecho o persigue a sus victimas silenciosamete, sobre todo al caer la noche.","Pato: El pato es un ave acuática, Anatidae. El pato tiene que ver con el ganso y el cisne. Hay patos en todos los colores y tamaños.","Perro: El perro es uno de los animales domésticos más antiguos del mundo y el mejor amigo del hombre. Su domesticación comienza cuando apenas era un pariente salvaje de los actuales lobos, existiendo pruebas arqueológicas que confirman su existencia como animal doméstico.","Pez: Los peces son animales vertebrados acuáticos, de respiración branquial, generalmente con extremidades en forma de aleta, aptas para la locomoción y sustentación en el agua. La piel, salvo raras excepciones, está protegida por escamas. Los peces son ovíparos, tienen esqueleto y se desplazan nadando. Son muy variados entre sí, pero por vivir en el agua presentan adaptaciones similares para nadar. Tienen sus cuerpos alargados, con los extremos anterior y posterior aguzados; esta forma del cuerpo facilita la natación.","Pingüino: Los pingüinos son aves no voladoras que viven en el hemisferio sur. De todas las aves que existen, son las mejor adaptadas al medio marino (viven el 75% de su vida en el agua). Tienen el cuerpo totalmente cubierto de pequeñas plumas y las alas convertidas en aletas planas, adaptadas a la natación, que no se pliegan y recubiertas de pequeñas plumas semejantes a escamas. Las patas cortas y colocadas muy atrás en el cuerpo; los dedos están unidos por una membrana. La cola es corta y está formada por plumas estrechas y tiesas. Anidan formando grandes colonias. Ponen uno o dos huevos de color blanco. Tienen una gruesa capa de grasa bajo la piel que los aísla de las bajas temperaturas que tienen que soportar tanto en el mar como en tierra. Su dieta está compuesta principalmente de crustáceos como el kril, peces, y cefalópodos como el calamar.","Puma: El puma, león de montaña, león o pantera es un mamífero carnívoro de la familia Felidae nativo de América.","Pulpo: El pulpo es un molusco marino, carnívoro. Carece de concha y posee ocho\nbrazos. ","Rana: La rana común, o verde, es un animal anfibio, que habita en las zonas de Europa y Asia que tienen un clima templado. Suelen ser de color verde o marrón con manchas negras y amarillentas por el dorso y más pálidas por el vientre.","Ratón: Mus es un género de roedores miomorfos de la familia Muridae que incluye la mayoría de los roedores llamados comúnmente ratones, si bien el nombre de ratón se usa para varias especies más pertenecientes a otros géneros.","Suricata: Suricata, o suricato, es el nombre de un pequeño mamífero, miembro de la familia de la mangosta, que habita la región del desierto del Kalahari y el Namib en África.","Salamandra: La salamandra común es una especie de anfibio urodelo (con cola). Es inconfundible por tener la piel de color negro con manchas variadas amarillas muy intensas que pueden llegar a cubrir la casi totalidad del cuerpo. A veces también se aprecian manchas de color rojizo.","Serpiente: Una serpiente es un animal vertebrado y reptil. Su nombre científico es serpentes. Tienen la piel cubierta de escamas. Además, mudan su piel cada cierto tiempo según van creciendo. Muchas especies usan veneno para inmovilizar o matar a sus presas.\nExisten más de 3.000 especies de serpientes.[3]","Tapir: El tapir es un animal de tamaño mediano, con una longitud que varía desde el 1,8 m hasta los 2,5 m, con una cola de 5 a 10 cm de largo, y una altura en la cruz de 70 cm a 1 m y un peso de 220 a 300 kg.","Tejón: El Tejón es un soberbio animal que es muy difícil verlo.Tiiene complexión robusta, cuerpo mucho más largo y ancho que alto, con patas cortas y cola también corta con el extremo romo. Durante el día, descansa en una gran madriguera situada a una profundidad de 3 a 5 metros.Esta madriguera tiene numerosas entradas normalmente esta habitada por un macho,una hembra y varios pequeños.el suelo de la madriguera esta recubierto de hierba ,de musgo ,de hojas y de otras plantas secas. Toda esta vegetación es renovada con regularidad pues el tejón es un animal limpio.Dispone incluso de un pequeño rincón ,cerca de su madriguera para hacer sus necesidades.Un agujero le sirve de baño si algún día llegas a descubrir unos aseos de este tipo ,sabrás que por ahí cerca vive algún tejón\nhábitat:toda Europa aunque escasea en ciertas regiones de Europa del norte.\nBiotopo:bosques con calveros \nAlimentos:insectos,lombrices,babosas,huevos,pajarillos,musarañas,nueces,raíces,animales muertos,ect.","Tiburón: El tiburón es un pez que habita principalmente en los mares de agua cálida. Es el predador más importante entre los peces por sus afilados dientes, que se reemplazan rápidamente a medida que los va perdiendo.","Tigre: Un tigre es un animal solitario y territorial que habita en los bosques densos y sabanas. Se encuentra solamente en el continente asiático; es un predador carnívoro y es la especie de felino más grande del mundo. Los tigres machos tienen un tamaño mucho mayor que el de las hembras."]

In [16]:
#TEXT TO SPEECH

def tts(_TEXTO):
    if __HABILITAR_TTS == True:
        # If service instance provides API key authentication
        service = TextToSpeechV1(
            ## url is optional, and defaults to the URL below. Use the correct URL for your region.
            url='https://stream.watsonplatform.net/text-to-speech/api',
            iam_apikey= __APIKEY_TEXT_TO_SPEECH)

        voices = service.list_voices().get_result()
        #print(json.dumps(voices, indent=2))

        with open(join(dirname('__file__'), __FOLDER + __FILE_TTS),
                  'wb') as audio_file:
            response = service.synthesize(_TEXTO, accept='audio/wav',
                voice="es-LA_SofiaVoice").get_result()
            audio_file.write(response.content)
        
        #Reproduce Sonido
        sound = AudioSegment.from_wav(__FOLDER+__FILE_TTS)
        play(sound)

In [17]:
#ASISTENTE
# If service instance provides API key authentication
assistant = AssistantV2(
    version='2018-09-20',
    ## url is optional, and defaults to the URL below. Use the correct URL for your region.
    url='https://gateway.watsonplatform.net/assistant/api',
    iam_apikey=__APIKEY_ASSISTANT)


def asistente(_TEXTO):
    
    #########################
    # Sessions
    #########################

    session = assistant.create_session(__ASSISTANT_ID).get_result()
    #print(json.dumps(session, indent=2))

    #########################
    # Message
    #########################

    message = assistant.message(
        __ASSISTANT_ID,
        session['session_id'],
        input={'text': _TEXTO},
        context={
            'metadata': {
                'deployment': 'myDeployment'
            }
        }).get_result()
    return (json.dumps(message, indent=2))

In [18]:

#GRABACION
FORMAT = pyaudio.paInt16
CHANNELS = 2
RATE = 44100
CHUNK = 1024
RECORD_SECONDS = 8
WAVE_OUTPUT_FILENAME = __FOLDER+__FILE_STT
 
def grabar():
    clear_output()
    beep()
    audio = pyaudio.PyAudio()

    # start Recording
    stream = audio.open(format=FORMAT, channels=CHANNELS,
                    rate=RATE, input=True,
                    frames_per_buffer=CHUNK)
    print('recording...')
    frames = []

    RANGE = int(RATE / CHUNK * RECORD_SECONDS)
    second = 0
    for i in range(0, RANGE):
        data = stream.read(CHUNK)
        frames.append(data)


        if i > 0:
            if second != int(i*RECORD_SECONDS / RANGE ):
                second = int(i*RECORD_SECONDS / RANGE )
                clear_output()
                #sleep(0.2) 
                print("◉ Grabando ("+str(RECORD_SECONDS - second)+") "+ ('* '*(RECORD_SECONDS - second)) )
        else:
            print("◉ Grabando ("+str(RECORD_SECONDS)+") "+ ('* '*(RECORD_SECONDS)) )

    clear_output()
    #print("finished recording")
    beep_finish()

    # stop Recording
    stream.stop_stream()
    stream.close()
    audio.terminate()

    waveFile = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
    waveFile.setnchannels(CHANNELS)
    waveFile.setsampwidth(audio.get_sample_size(FORMAT))
    waveFile.setframerate(RATE)
    waveFile.writeframes(b''.join(frames))
    waveFile.close()

In [19]:

#Speech to Text
class MyRecognizeCallback(RecognizeCallback):
    def __init__(self):
        RecognizeCallback.__init__(self)

    def on_transcription(self, transcript):
        print('transcript')
        print(transcript)

    def on_connected(self):
        #print('Connection was successful')
        print('')

    def on_error(self, error):
        print('Error received: {}'.format(error))

    def on_inactivity_timeout(self, error):
        print('Inactivity timeout: {}'.format(error))

    def on_listening(self):
        #print('Service is listening')
        print('')

    def on_hypothesis(self, hypothesis):
        print('hypothesis')
        print(hypothesis)

    def on_data(self, data):
        print('data')
        print(data)

def stt():
    # If service instance provides API key authentication
    service = SpeechToTextV1(
        ## url is optional, and defaults to the URL below. Use the correct URL for your region.
        url='https://stream.watsonplatform.net/speech-to-text/api',
        iam_apikey=__APIKEY_SPEECH_TO_TEXT)

    with open(join(dirname('__file__'), __FOLDER+__FILE_STT),
              'rb') as audio_file:
        result = json.dumps(
            service.recognize(
                audio=audio_file,
                content_type='audio/wav',
                timestamps=True,
                word_confidence=True,
                model= 'es-ES_BroadbandModel'
            ).get_result(),
            indent=2)
        j = json.loads(result)
        texto = ""
        try:
            texto = j['results'][0]['alternatives'][0]['transcript']
        except:
            texto = ""
        print(texto)
        return texto

    # Example using threads in a non-blocking way
    mycallback = MyRecognizeCallback()
    audio_file = open(join(dirname('__file__'), __FOLDER+__FILE_STT), 'rb')
    audio_source = AudioSource(audio_file)
    recognize_thread = threading.Thread(
        target=service.recognize_using_websocket,
        args=(audio_source, "audio/l16; rate=44100", mycallback))
    recognize_thread.start()

In [20]:
#FUNCIONES
def saludo():
    print("Yo soy Alicia, y soy tu asistente personal")
    
    if __HABILITAR_TTS == True:
        #convertir la respuesta a audio
        saludo = '<p><s> <prosody rate="+15%">¡Hola!</prosody> <break time="100ms"/> <prosody rate="+5%"> Yo soy Alicia, y soy tu asistente personal</prosody> ¿Dime qué deseas realizar?</s></p>'
        tts(saludo)
    else:
        time.sleep(4)

In [21]:
def beep():
    #reproducir la respuesta
    beep = AudioSegment.from_wav(__FOLDER+__FILE_BEEP)
    play(beep)

def beep_finish():
    #reproducir la respuesta
    beep = AudioSegment.from_wav(__FOLDER+__FILE_BEEP_FINISH)
    play(beep)

In [22]:
def imc(peso, estatura):
    
    IMC=peso/(estatura/100)**2
    
    provoca = ""
    tipo = "" 
    mensaje = ""
    recomendacion = ""
    diagnostico = ""
    
    if IMC < 5:
        tipo = "Delgadez 3"
        diagnostico = "Delgadez de nivel 3, tienes un peso demasiado bajo"
        provoca = "te puede causar: Postración, Atenia, Adinamia, Enfermedades Degenerativas y Peligro de Muerte."
    elif IMC >= 5 and IMC < 10:
        tipo = "Delgadez 2"
        diagnostico = "Delgadez de nivel 2, tienes un peso muy bajo"
        provoca = "te puede causar: Anorexia, Bulimia, Osteoporosis y Autoconsumo de Masa Muscular."
    elif IMC >= 10 and IMC < 18.5:
        tipo = "Delgadez 1"
        diagnostico = "Delgadez de nivel 1, tienes un peso bajo"
        provoca = "te puede causar: Trastornos Digestivos, Debilidad, Fatiga Crónica, Estrés, Ansiedad y Delifusión Hormonal."
    elif IMC >= 18.5 and IMC < 25:
        tipo = "Peso Normal"
        diagnostico = "Tu peso es normal, felicidades"
        provoca = "tienes un Estado Normal, Buen Nivel de Energía, Vitalidad y Buena Condición Física."
    elif IMC >= 25 and IMC < 30:
        tipo = "Sobrepeso"
        diagnostico = "Tienes sobrepeso, vigila tu dieta"
        provoca = "te puede causar: Fatiga, Enfermedades Digestivas, Problemas Cardíacos, Mala Circulación en piernas y Várices."
    elif IMC >= 30 and IMC < 35:
        tipo = "Obesidad 1"
        diagnostico = "Obesidad nivel 1, consulta a tu médico"
        provoca = "te puede causar: Diabetes, Hipertensión, Enfermedades Cardiovasculares, Problemas Articulares, Rodilla y Columna, Cálculos Biliares."
    elif IMC >= 35 and IMC < 40:
        tipo = "Obesidad 2"
        diagnostico = "Obesidad nivel 2, consulta a tu médico"
        provoca = "te puede causar: Diabetes, Cáncer, Angina de Pecho, Infartos, Tromboflebitis, Arterosclerosis, Embolias, Alteraciones de Menstruación."
    elif IMC>=40:
        tipo = "Obesidad 3"
        diagnostico = "Obesidad grave de nivel 3, consulta a tu médico"
        provoca = "te puede causar: Falta de Aire, Somnolencia, Trombosis Pulmonar, Úlceras Varicosas, Cáncer de Próstata, Reflujo Esofágico, Discriminación Social, Laboral y Sexual."
    
    peso_ideal = estatura -100 - ( (estatura - 150) / 4 )
    recomendacion = ""
    if IMC <18.5 or IMC >=25:
        recomendacion = "Tu peso ideal es de aproximadamente {:.0f} kg te recomiendo {} de {:.0f} a {:.0f} kg".format(peso_ideal, 'bajar' if peso-peso_ideal >= 0  else 'subir', abs(peso-peso_ideal) - 4 ,abs(peso-peso_ideal))
    
    return ("{}, {} {} ".format(diagnostico, provoca, recomendacion))

In [23]:
def iniciar_alicia():
    #Bienvenida
    saludo()
    
    cont_error = 0
    terminar = False
    
    while terminar == False:
        
        if __HABILITAR_STT == True:
            #grabar del microfono
            grabar()

            print("Procesando Grabación...")
            #convertir la grabación a texto
            texto = stt()
        else:
            texto    = input("¿Qué deseas realizar?: ")

        clear_output()
        print('Tú: '+texto)

        #verificar en el asistente la pregunta
        respuesta_asistente = "Sin Respuesta"
        try:
            respuesta_asistente = asistente(texto)
        except:
            session = assistant.create_session(__ASSISTANT_ID).get_result()
            respuesta_asistente = asistente(texto)

        json_asistente = json.loads(respuesta_asistente)
        try:
            asistente_generic = json_asistente['output']['generic'][0]['text']            
        except:
            asistente_generic = ""
            
        try:
            asistente_intent = json_asistente['output']['intents'][0]['intent']
        except:
            asistente_generic = "Lo siento, no entendí muy bien, ¿Qué deseas realizar?"
            asistente_intent = None
            cont_error = cont_error +1

        
             
        if cont_error == 3:
            respuesta = '''<p><s><prosody rate="+5%"> '''+'Adios'+'''</prosody></s></p>'''
            terminar = True
            print('Alicia: '+ 'Adios')
        
        
        #convertir la respuesta a audio
        if asistente_intent == 'salir':
            respuesta = '''<p><s><prosody rate="+5%"> '''+asistente_generic+'''</prosody></s></p>'''
            print('Alicia: '+ asistente_generic)
            terminar = True
        elif asistente_intent == 'topico_calculo':            
            cont_error = 0
            texto = topicos[random.randint(1,len(topicos))]
            respuesta = '''<p><s> <prosody rate="+15%">¡Muy Bien!, te diré un Tópico de Cálculo</prosody> <break time="500ms"/> 
                <prosody rate="+5%"> '''+texto+'''</prosody>
                ¿Qué más deseas realizar?</s></p>'''
            print('Alicia: '+ texto)
        elif asistente_intent == 'animales':            
            cont_error = 0
            texto = animales[random.randint(1,len(animales))]
            respuesta = '''<p><s> <prosody rate="+15%">¡Muy Bien!, te diré un Animal</prosody> <break time="500ms"/> 
                <prosody rate="+5%"> '''+texto+'''</prosody>
                ¿Qué más deseas realizar?</s></p>'''
            print('Alicia: '+ texto)
        elif asistente_intent == 'la_hora':            
            cont_error = 0 
            now_utc = datetime.now(timezone('UTC'))
            now_mexico = now_utc.astimezone(timezone('America/Mexico_City'))
            texto = now_mexico.strftime("son las %H horas con %M minutos")
            respuesta = '''<p><s> <prosody rate="+15%">¡Muy Bien!</prosody> <break time="500ms"/> 
                <prosody rate="+5%"> '''+texto+'''</prosody>
                ¿Qué más deseas realizar?</s></p>'''
            print('Alicia: '+ texto)
        elif asistente_intent == 'calcular_imc':            
            cont_error = 0 
            texto = asistente_generic
            respuesta = '''<p><s> 
                <prosody rate="+5%"> '''+texto+'''</prosody>
                </s></p>'''
            print('Alicia: '+ texto)
        elif asistente_intent == 'imc':            
            cont_error = 0 
            
            try:
                e1 = int(json_asistente['output']['entities'][0]['value'])
                e2 = int(json_asistente['output']['entities'][1]['value'])
                
                peso = -1
                estatura = -1

                if e1 > e2:
                    estatura = e1
                    peso = e2
                else:
                    peso = e1
                    estatura = e2
                    
                texto = "Muy Bien, mides "+str(estatura)+" cm y pesas "+str(peso)+" kg. "+imc(peso, estatura)
            except:
                texto = 'Lo siento, no entendí muy bien, ¿Qué deseas realizar?'
            
            respuesta = '''<p><s>
                <prosody rate="+5%"> '''+texto+'''</prosody>
                ¿Qué más deseas realizar?</s></p>'''
            print('Alicia: '+ texto)
        elif asistente_intent:
            cont_error = 0
            respuesta = '''<p><s> <prosody rate="+15%">¡Muy Bien!</prosody> <break time="500ms"/> 
                <prosody rate="+5%"> '''+asistente_generic+'''</prosody>
                ¿Qué más deseas realizar?</s></p>'''
            print('Alicia: '+ asistente_generic)
        else:
            respuesta = '''<p><s><prosody rate="+5%"> '''+asistente_generic+'''</prosody></s></p>'''
            print('Alicia: '+ asistente_generic)
       
            
        #reproducir la respuesta
        tts(respuesta)

# ALICIA
## Watson
#### Speech to Text, Text to Speech y Assistant

### IMC
calcula mi imc, como esta mi peso, cual es mi imc, @dime como esta mi peso, @dime imc, @dime mi imc, evalúa mi peso, imc

mido @sys-number centímetros y peso @sys-number kilos, peso @sys-number kilos y mido @sys-number centímetros, @sys-number centímetros @sys-number kilos, @sys-number centímetros y @sys-number kilos, @sys-number kilos y mido @sys-number centímetros, @sys-number kilos y @sys-number centímetros

### Tópico de Cálculo
@dime otro tópico, @dime otro tópico de cálculo, @dime tópico, @dime tópico de cálculo, @dime un tópico, @dime un tópico de cálculo, otro tópico, un tópico de cálculo

### La Hora
@dime la hora, @dime que hora es, la hora, que hora es, que hora son, que horas son

### Animales
animal aleatorio, @dime un animal, @dime un animal aleatorio, un animal

### Otros
adios, bye, cerrar, detente, @dime salir, eso sería todo, para, parar, salir
ayuda, ayudame, como funciona, como funciona esto, @dime ayuda, help, quiero saber como funciona, repiteme la ayuda

In [26]:
iniciar_alicia()

Tú: adios
Alicia: Gracias por tu tiempo. ¡Adiós! 
